In [1]:
from datasets import load_dataset, Value, Features, ClassLabel

In [2]:
data_path = "./data/clue/tnews/"


labels = load_dataset("json", data_files=data_path + "labels.json")
labels = labels['train']['label']

features = Features({"label": ClassLabel(num_classes=len(labels), names=labels), "sentence": Value(dtype="string", id=None)})


input_dataset = load_dataset("json", data_files={"train": data_path + "train.json", "dev": data_path + "dev.json"})
test_dataset = load_dataset("json", data_files={"test": data_path + "test.json"})
input_dataset = input_dataset.cast_column("label", ClassLabel(names=labels))

Using custom data configuration default-540fc36db350309f
Found cached dataset json (/home/mymusise/.cache/huggingface/datasets/json/default-540fc36db350309f/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 1/1 [00:00<00:00, 608.05it/s]
Using custom data configuration default-b2755cebdcb7b60d
Found cached dataset json (/home/mymusise/.cache/huggingface/datasets/json/default-b2755cebdcb7b60d/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 2/2 [00:00<00:00, 484.44it/s]
Using custom data configuration default-71a9be806c0c3603
Found cached dataset json (/home/mymusise/.cache/huggingface/datasets/json/default-71a9be806c0c3603/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 1/1 [00:00<00:00, 784.13it/s]
Loading cached processed dataset at /home/mymusise/.cache/huggingface/datasets/json/default-b2755cebdcb7b60d/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096c

In [2]:
from transformers import AutoTokenizer


model_path = "/data2/CPM-distill/CPM-Generate-distill"

tokenizer = AutoTokenizer.from_pretrained(model_path)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CpmTokenizer'. 
The class this function is called from is 'XLNetTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
def token_map(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=32, add_special_tokens=False)

train_dataset = input_dataset['train'].map(token_map, batched=True)
eval_dataset = input_dataset['dev'].map(token_map, batched=True)

  0%|          | 0/54 [00:00<?, ?ba/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.459 seconds.
Prefix dict has been built successfully.
100%|██████████| 10/10 [00:01<00:00,  9.50ba/s]


In [5]:
train_dataset = train_dataset.shuffle(seed=42)
eval_dataset = eval_dataset.shuffle(seed=42)

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=len(labels)+1)

Some weights of the model checkpoint at /data2/CPM-distill/CPM-Generate-distill were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at /data2/CPM-distill/CPM-Generate-distill and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# ensure some model padding token id
model.config.pad_token_id = tokenizer.pad_token_id

In [8]:
import numpy as np
import evaluate


metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=16)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
print(f"before train: {trainer.evaluate()}")
trainer.train()
print(f"after train: {trainer.evaluate()}")

The following columns in the evaluation set  don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: label_desc, sentence, keywords. If label_desc, sentence, keywords are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
100%|██████████| 1250/1250 [00:12<00:00, 98.60it/s] 
The following columns in the training set  don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: label_desc, sentence, keywords. If label_desc, sentence, keywords are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
/home/mymusise/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_depr

before train: {'eval_loss': 2.926095485687256, 'eval_accuracy': 0.0727, 'eval_runtime': 12.6987, 'eval_samples_per_second': 787.483, 'eval_steps_per_second': 98.435}


  5%|▍         | 500/10005 [00:23<08:38, 18.33it/s]Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json


{'loss': 1.9604, 'learning_rate': 4.750124937531234e-05, 'epoch': 0.15}


Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
 10%|▉         | 1000/10005 [00:50<07:35, 19.79it/s]Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json


{'loss': 1.6512, 'learning_rate': 4.500249875062469e-05, 'epoch': 0.3}


Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
 15%|█▍        | 1500/10005 [01:17<07:40, 18.48it/s]Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json


{'loss': 1.6015, 'learning_rate': 4.250374812593703e-05, 'epoch': 0.45}


Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
